In [27]:
import requests
from bs4 import BeautifulSoup
import hashlib
import os
import io
import zipfile

from azure.storage.blob import BlobServiceClient
from concurrent.futures import ThreadPoolExecutor

In [28]:
# URL of the webpage containing the .csv files
url = 'http://data.gdeltproject.org/events/index.html'  # Replace with the actual URL
base_url = 'http://data.gdeltproject.org/events/'

In [29]:
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
files = soup.find_all('a', href=True)
# # Example of parsing the webpage to find the links, sizes, and MD5s
# files = soup.find_all('a', href=True)

file_lists = []
for file in files:
    # print(file['href'])
    if file['href'].endswith('.zip'):
        file_lists.append(base_url+file['href'])

In [34]:
# Azure Storage account details
import io

storage_account_name = "factoredatathon"
storage_account_key = "yDTqsi+AifQJPvC5r7L5iFFdmmj+fbxWr280etWbWMPXWij0yfmiuLJH3sZ91TI7SwmfR1SBD8L7+AStGVUo3Q=="
container_name = "bronze"

# Create a BlobServiceClient
blob_service_client = BlobServiceClient(
    f"https://{storage_account_name}.blob.core.windows.net", storage_account_key
)

# Function to download and upload a file
def download_and_upload(url):
    try:
        # Download the file
        file_name = os.path.basename(url)
        response = requests.get(url)
        local_path = f"/tmp/{file_name}"
        
        # Decompress the .zip file in memory
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            for zip_info in z.infolist():
                # Extract the file to a byte stream
                with z.open(zip_info) as extracted_file:
                    # Upload the extracted file to Azure Data Lake Storage
                    blob_name = zip_info.filename  # Use the name of the file inside the .zip
                    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
                    blob_client.upload_blob(extracted_file.read(), overwrite=True)
                    print(f"Uploaded {blob_name} to Azure Data Lake Storage")

    except Exception as e:
        print(f"Failed to upload {file_name}: {e}")
        
# Use ThreadPoolExecutor to download and upload files concurrently
with ThreadPoolExecutor() as executor:
    executor.map(download_and_upload, file_lists[1:6])

Uploaded 20240811.export.CSV to Azure Data Lake Storage
Uploaded 20240812.export.CSV to Azure Data Lake Storage
Uploaded 20240809.export.CSV to Azure Data Lake Storage
Uploaded 20240813.export.CSV to Azure Data Lake Storage
Uploaded 20240810.export.CSV to Azure Data Lake Storage


In [44]:
count = 0
for file in file_lists:
    count =+ 1
    print(count)
    download_and_upload(file)

1
Uploaded 20240812.gkg.csv.zip to Azure Data Lake Storage
1
Uploaded 20240812.gkgcounts.csv.zip to Azure Data Lake Storage
1
Uploaded 20240811.gkg.csv.zip to Azure Data Lake Storage
1
Uploaded 20240811.gkgcounts.csv.zip to Azure Data Lake Storage
1
Uploaded 20240810.gkg.csv.zip to Azure Data Lake Storage
1
Uploaded 20240810.gkgcounts.csv.zip to Azure Data Lake Storage
1
Uploaded 20240809.gkg.csv.zip to Azure Data Lake Storage
1
Uploaded 20240809.gkgcounts.csv.zip to Azure Data Lake Storage
1
Uploaded 20240808.gkg.csv.zip to Azure Data Lake Storage
1
Uploaded 20240808.gkgcounts.csv.zip to Azure Data Lake Storage
1
Uploaded 20240807.gkg.csv.zip to Azure Data Lake Storage
1
Uploaded 20240807.gkgcounts.csv.zip to Azure Data Lake Storage
1
Uploaded 20240806.gkg.csv.zip to Azure Data Lake Storage
1
Uploaded 20240806.gkgcounts.csv.zip to Azure Data Lake Storage
1
Uploaded 20240805.gkg.csv.zip to Azure Data Lake Storage
1
Uploaded 20240805.gkgcounts.csv.zip to Azure Data Lake Storage
1
Upload

KeyboardInterrupt: 

In [28]:
file_lists[0:10]

['http://data.gdeltproject.org/gkg/20240812.gkg.csv.zip',
 'http://data.gdeltproject.org/gkg/20240812.gkgcounts.csv.zip',
 'http://data.gdeltproject.org/gkg/20240811.gkg.csv.zip',
 'http://data.gdeltproject.org/gkg/20240811.gkgcounts.csv.zip',
 'http://data.gdeltproject.org/gkg/20240810.gkg.csv.zip',
 'http://data.gdeltproject.org/gkg/20240810.gkgcounts.csv.zip',
 'http://data.gdeltproject.org/gkg/20240809.gkg.csv.zip',
 'http://data.gdeltproject.org/gkg/20240809.gkgcounts.csv.zip',
 'http://data.gdeltproject.org/gkg/20240808.gkg.csv.zip',
 'http://data.gdeltproject.org/gkg/20240808.gkgcounts.csv.zip']

In [14]:
# Function to scrape the webpage and download files
def scrape_and_download(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Example of parsing the webpage to find the links, sizes, and MD5s
    files = soup.find_all('a', href=True)
    for file in files:
        if file['href'].endswith('.csv.zip'):
            file_url = file['href']
            file_name = os.path.basename(file_url)
            save_path = os.path.join(os.getcwd(), file_name)
            
            # Extract the corresponding file size and MD5 (pseudo-code, depends on the actual HTML structure)
            file_size = 'size'  # Replace with actual extraction logic
            md5sum = 'md5'      # Replace with actual extraction logic

            # Download the file
            download_file(file_url, save_path)
            print(f'Downloaded: {file_name}')

            # Verify MD5 checksum
            downloaded_md5 = calculate_md5(save_path)
            if downloaded_md5 == md5sum:
                print(f'MD5 checksum verified for {file_name}')
            else:
                print(f'Warning: MD5 checksum mismatch for {file_name}')
            

In [15]:
# Execute the function
scrape_and_download(url)

MissingSchema: Invalid URL '20240812.gkg.csv.zip': No scheme supplied. Perhaps you meant https://20240812.gkg.csv.zip?